In [23]:
# PAGE MACHINE LEARNING

from tkinter import *
from tkinter.filedialog import askopenfilename
import csv
import os
from tkinter import ttk
import time
import pandas as pd
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tkinter.messagebox as tkmessagebox
from PIL import ImageTk, Image

gui = Tk()
width_value = gui.winfo_screenwidth()
height_value = gui.winfo_screenheight()

bg = Image.open(r"malware.png") 
bg = bg.resize((width_value,height_value), Image.ANTIALIAS)
test_bg = ImageTk.PhotoImage(bg)
label = Label(image=test_bg, width=width_value, height=height_value)
label.image = test_bg
label.place(x=0, y=0)

gui.title('Machine Learning GUI')
gui.geometry('{}x{}+0+0'.format(width_value,height_value))
gui.rowconfigure([0,1,2,3,4,5], weight=0)

def data(): # Upload File Data CSV ke Tkinter
    global filename
    filename = askopenfilename(initialdir='C:\\',title = "Select file")
    e1.delete(0, END)
    e1.insert(0, filename)
    #e1.config(text=filename)
    #print(filename)
    global df
    global used_data
    try:
        df = pd.read_csv(filename)
        used_data = df.drop(['Name', 'Machine', 'TimeDateStamp', 'Malware'], axis=1)
    except ValueError:
        tkmessagebox.showerror("Information", "Format file yang dimasukkan salah!")
        return None
    except FileNotFoundError:
        tkmessagebox.showerror("Information", "Tidak ada file seperti {}".format(filename))
        return None
    clear_data()
    file['column'] = list(used_data.columns)
    file['show'] = "headings"
    for column in file['columns']:
        file.heading(column, text=column)
    df_rows = used_data.to_numpy().tolist()
    for row in df_rows:
        file.insert("", "end", values=row)
    return None
    
    
def clear_data():
    file.delete(*file.get_children())

def ML():
    l2=Label(hasil_box, text='Pembagian Data:')
    l2.grid(row=0, column=0, sticky='ns', pady=20, padx=10)
    f = Figure(figsize=(10,4), dpi=70)
    ax = f.subplots()
    ax.set_xticklabels(['Benign', 'Malware'])
    sns.countplot(df['Malware'], ax=ax)
    canvas = FigureCanvasTkAgg(f, master=hasil_box)
    canvas.draw()
    canvas.get_tk_widget().grid(row=0, column=1, sticky='n', rowspan=4)
#     toolbar = NavigationToolbar2Tk(canvas,gui)
#     canvas._tkcanvas.grid(column=3, row=1)
    X_train, X_test, y_train, y_test = train_test_split(used_data, df['Malware'], test_size=0.2, random_state=0)
    rfc = RandomForestClassifier(n_estimators=100, random_state=0, 
                         oob_score = True,
                         max_depth = 16)
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    report = classification_report(y_test, y_pred, target_names=['Benign', 'Malware'])
#     l3=Label(hasil_box, textvariable=report)
#     l3.grid(row=1, column=0, sticky='w', pady=20, padx=10)
    score = str('{:.2f}'.format(accuracy_score(y_test,y_pred)*100)) + " %"
    var = StringVar()
    var.set(score)
    l5=Label(hasil_box, text='Confusion Matrix Data: ')
    l5.grid(row=6, column=0, sticky='ns', pady=20, padx=10)
    f2 = Figure(figsize=(10,4), dpi=70)
    ax2 = f2.subplots()
    ax2=sns.heatmap(confusion_matrix(y_pred, y_test), annot=True, fmt="d", cmap=plt.cm.Blues, cbar=False)
    ax2.set_xlabel('Predicted labels')
    ax2.set_ylabel('True labels')
    canvas2 = FigureCanvasTkAgg(f2, master=hasil_box)
    canvas2.draw()
    canvas2.get_tk_widget().grid(row=6, column=1, sticky='ns', rowspan=4, pady=10)
    l3=Label(hasil_box, text='Hasil Akurasi Random Forest: ')
    l3.grid(row=10, column=0, sticky='w', pady=20, padx=10)
    l4=Label(hasil_box, textvariable=var)
    l4.grid(row=10, column=1, sticky='w', pady=20, padx=10)
    
# INPUT DATAFRAME
l1=Label(gui, text='Select Data File')
l1.grid(row=0, column=0, sticky='E', pady=20, padx=10)

e1 = Entry(gui,text='')
e1.grid(row=0, column=1, sticky='W', ipadx=100)

Button(gui,text='open', command=data).grid(row=0, column=2, sticky="W", ipadx=10)

# TAMPILAN DATAFRAME
file_box = LabelFrame(gui,text="dataframe") 
file_box.grid(ipadx=350, ipady=300, padx=10, row=1, columnspan=3)
file = ttk.Treeview(file_box)
file.place(relheight=1, relwidth=1)
filescrolly = Scrollbar(file_box, orient="vertical", command=file.yview)
filescrollx = Scrollbar(file_box, orient="horizontal", command=file.xview)
file.configure(xscrollcommand=filescrollx.set, yscrollcommand=filescrolly.set)
filescrollx.pack(side='bottom', fill='x')
filescrolly.pack(side='right', fill='y')

# PROGRESS BAR
progress_bar = ttk.Progressbar(orient = 'horizontal', length=765, mode='determinate')
progress_bar.grid(row=2, columnspan=3, pady = 10)

# BUTTON PROSES
proses = Button(gui, text='Proses', command=ML)
proses.grid(row=3, columnspan=3, ipadx=20)

# TAMPILAN HASIL PROSES MACHINE LEARNING
hasil_box = LabelFrame(gui,text="Hasil") 
hasil_box.grid(ipadx=500, ipady=400, row=0, column=3, rowspan=10, sticky='NSEW')
hasil_box.rowconfigure(0, weight=0)
hasil_box.rowconfigure(1, weight=0)

gui.mainloop()

<ipython-input-23-1ef01177fffb>:73: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['Benign', 'Malware'])
E:\Users\fauzanhikmahramadhan\Anaconda3\envs\python388\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
